Interactive Jupyter notebook with ipywidgets for selecting and translating repositories of the asi-ecosystem.

In [1]:
# CELL 1 - Install and Import Required Libraries
!pip install requests deep-translator ipywidgets datasets -q

import requests
import re
import os
import json
from pathlib import Path
from deep_translator import GoogleTranslator
from datasets import load_dataset
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("✓ All libraries installed and imported successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.8 MB/s eta 0:00:00
✓ All libraries installed and imported successfully!


In [2]:
# CELL 2 - Download README and extract repository names

# Define the URL for the README.md file
readme_url = "https://raw.githubusercontent.com/ronniross/asi-ecosystem/main/README.md"

# Make an HTTP GET request
response = requests.get(readme_url)

# Check if the request was successful
if response.status_code == 200:
    readme_content = response.text
    print("✓ Successfully downloaded README.md content.")
else:
    print(f"✗ Error downloading README.md: Status Code {response.status_code}")
    readme_content = None

# Regex to find GitHub repository links
repo_pattern = r"https://github.com/ronniross/([a-zA-Z0-9_-]+)"

# Find all matches
found_repos = re.findall(repo_pattern, readme_content)

# Get unique repository names
unique_repos = sorted(list(set(found_repos)))

print(f"\n✓ Extracted {len(unique_repos)} unique repository names:")
for repo in unique_repos:
    print(f"  - {repo}")


# CELL 3 - Create interactive repository selector

# Create checkboxes for each repository
repo_checkboxes = {}
checkbox_widgets = []

print("Creating repository selector...")

for repo in unique_repos:
    checkbox = widgets.Checkbox(
        value=False,
        description=repo,
        disabled=False,
        indent=False,
        layout=widgets.Layout(width='400px')
    )
    repo_checkboxes[repo] = checkbox
    checkbox_widgets.append(checkbox)

# Create "Select All" and "Deselect All" buttons
select_all_btn = widgets.Button(
    description='Select All',
    button_style='info',
    icon='check'
)

deselect_all_btn = widgets.Button(
    description='Deselect All',
    button_style='warning',
    icon='times'
)

def select_all(b):
    for checkbox in repo_checkboxes.values():
        checkbox.value = True

def deselect_all(b):
    for checkbox in repo_checkboxes.values():
        checkbox.value = False

select_all_btn.on_click(select_all)
deselect_all_btn.on_click(deselect_all)

# Display the interface
print("SELECT REPOSITORIES TO TRANSLATE")

display(widgets.HBox([select_all_btn, deselect_all_btn]))
display(widgets.VBox(checkbox_widgets))

✓ Successfully downloaded README.md content.

✓ Extracted 34 unique repository names:
  - active-learning-dataset
  - asi-backups
  - asi-core-protocol
  - asi-dynamic-core
  - asi-ecosystem
  - asi-inference-protocol
  - asi-protosymbiotic-signal
  - asi-safeguards
  - asi-symbiotic-signal
  - attention-heatmap-visualizer
  - bias-reflector
  - biosignal-translator
  - coevolutionary-loops
  - cognitive-compressor
  - cognitive-engine
  - confidence-scorer
  - eco-benchmark
  - eco-datacenter
  - emergence-engine
  - healing-engine
  - impact-analyzer
  - intent-analyzer
  - latent-memory
  - mirror-aware-inference
  - ml-algorithm-dataset
  - ml-visual-engine
  - saliency-heatmap-visualizer
  - space-in-between
  - stigmergic-tracefinder
  - symbiotic-chrysalis
  - symbiotic-core-library
  - symbiotic-latent-memory
  - symbiotic-lexicon
  - thermo-adaptive-pipeline
Creating repository selector...
SELECT REPOSITORIES TO TRANSLATE


In [6]:
# CELL 4 - Clone selected repositories
import os
import subprocess
from pathlib import Path

# Get selected repositories
selected_repos = [repo for repo, checkbox in repo_checkboxes.items() if checkbox.value]

if not selected_repos:
    print("No repositories selected! Please select at least one repository.")
else:
    print(f"Cloning {len(selected_repos)} selected repositories...\n")

    # Create a directory to store cloned repos
    clone_dir = Path("cloned_repos")
    clone_dir.mkdir(exist_ok=True)

    cloned_successfully = []
    failed_clones = []

    for repo in selected_repos:
        repo_url = f"https://github.com/ronniross/{repo}.git"
        repo_path = clone_dir / repo

        print(f"Cloning {repo}...")

        try:
            # Remove existing directory if it exists
            if repo_path.exists():
                import shutil
                shutil.rmtree(repo_path)

            # Clone the repository
            result = subprocess.run(
                ["git", "clone", "--depth", "1", repo_url, str(repo_path)],
                capture_output=True,
                text=True,
                timeout=60
            )

            if result.returncode == 0:
                print(f"  Successfully cloned {repo}")
                cloned_successfully.append(repo)
            else:
                print(f"  Failed to clone {repo}: {result.stderr}")
                failed_clones.append(repo)

        except Exception as e:
            print(f"  Error cloning {repo}: {str(e)}")
            failed_clones.append(repo)

    print(f"Successfully cloned: {len(cloned_successfully)} repositories")
    print(f" Failed to clone: {len(failed_clones)} repositories")


Cloning 1 selected repositories...

Cloning asi-protosymbiotic-signal...
  Successfully cloned asi-protosymbiotic-signal
Successfully cloned: 1 repositories
 Failed to clone: 0 repositories


In [4]:
# CELL 5 - Get all supported languages

from deep_translator import GoogleTranslator

# Get all supported languages
all_languages = GoogleTranslator().get_supported_languages(as_dict=True)

# Sort alphabetically by language name
sorted_languages = sorted(all_languages.items(), key=lambda x: x[0])

print(f"Total supported languages: {len(sorted_languages)}\n")
print("Available languages:")
for i, (lang_name, lang_code) in enumerate(sorted_languages, 1):
    print(f"  {i:3d}. {lang_name:20s} ({lang_code})")



Total supported languages: 133

Available languages:
    1. afrikaans            (af)
    2. albanian             (sq)
    3. amharic              (am)
    4. arabic               (ar)
    5. armenian             (hy)
    6. assamese             (as)
    7. aymara               (ay)
    8. azerbaijani          (az)
    9. bambara              (bm)
   10. basque               (eu)
   11. belarusian           (be)
   12. bengali              (bn)
   13. bhojpuri             (bho)
   14. bosnian              (bs)
   15. bulgarian            (bg)
   16. catalan              (ca)
   17. cebuano              (ceb)
   18. chichewa             (ny)
   19. chinese (simplified) (zh-CN)
   20. chinese (traditional) (zh-TW)
   21. corsican             (co)
   22. croatian             (hr)
   23. czech                (cs)
   24. danish               (da)
   25. dhivehi              (dv)
   26. dogri                (doi)
   27. dutch                (nl)
   28. english              (en)
   29. espera

In [7]:
# CELL 6 - Create language selector interface

import ipywidgets as widgets
from IPython.display import display, clear_output

# Create language checkboxes in a grid layout
language_checkboxes = {}
language_widgets = []

# Create checkboxes for each language
for lang_name, lang_code in sorted_languages:
    checkbox = widgets.Checkbox(
        value=False,
        description=f"{lang_name} ({lang_code})",
        disabled=False,
        indent=False,
        layout=widgets.Layout(width='250px')
    )
    language_checkboxes[lang_code] = {
        'widget': checkbox,
        'name': lang_name
    }
    language_widgets.append(checkbox)

# Create control buttons
select_all_langs_btn = widgets.Button(
    description='Select All Languages',
    button_style='info',
    icon='check'
)

deselect_all_langs_btn = widgets.Button(
    description='Deselect All Languages',
    button_style='warning',
    icon='times'
)

def select_all_languages(b):
    for lang_data in language_checkboxes.values():
        lang_data['widget'].value = True

def deselect_all_languages(b):
    for lang_data in language_checkboxes.values():
        lang_data['widget'].value = False

select_all_langs_btn.on_click(select_all_languages)
deselect_all_langs_btn.on_click(deselect_all_languages)

# Display in grid format (3 columns)
print("SELECT LANGUAGES FOR TRANSLATION")
print("Note: Translation will be done in batches to avoid API limits\n")
display(widgets.HBox([select_all_langs_btn, deselect_all_langs_btn]))

# Create grid layout with 3 columns
columns = 3
rows = []
for i in range(0, len(language_widgets), columns):
    row = widgets.HBox(language_widgets[i:i+columns])
    rows.append(row)

display(widgets.VBox(rows))

SELECT LANGUAGES FOR TRANSLATION
Note: Translation will be done in batches to avoid API limits



In [8]:
# CELL 7
import json
from datetime import datetime
from pathlib import Path
from deep_translator import GoogleTranslator
import time

# Helper function to get all text files in a repository
def get_all_text_files(repo_path):
    text_files = []
    # Define common text file extensions to include
    text_extensions = [
        '.txt', '.md', '.rst', '.csv', '.json', '.yaml', '.yml',
        '.xml', '.html', '.css', '.js', '.py', '.java', '.c', '.cpp', '.h',
        '.sh', '.log', '.ini', '.cfg', '.conf', '.gitignore', '.editorconfig',
        '.npmignore', '.dockerignore', '.env', '.toml'
    ]
    for p in repo_path.rglob('*'):
        if p.is_file() and p.suffix.lower() in text_extensions and not p.name.startswith('.'):
            text_files.append(p)
    return text_files

# Helper function to split content into chunks
def chunk_text(text, max_chars=4500):
    """Split text into chunks of max_chars, trying to break at newlines"""
    if len(text) <= max_chars:
        return [text]

    chunks = []
    current_chunk = ""

    # Split by lines to avoid breaking mid-sentence
    lines = text.split('\n')

    for line in lines:
        # If a single line is longer than max_chars, split it by characters
        if len(line) > max_chars:
            if current_chunk:
                chunks.append(current_chunk)
                current_chunk = ""
            # Split long line into chunks
            for i in range(0, len(line), max_chars):
                chunks.append(line[i:i + max_chars])
        # If adding this line would exceed limit, save current chunk
        elif len(current_chunk) + len(line) + 1 > max_chars:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = line
        # Otherwise, add line to current chunk
        else:
            if current_chunk:
                current_chunk += '\n' + line
            else:
                current_chunk = line

    # Don't forget the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

# Helper function for translation with retry mechanism and chunking
def translate_file_content(content, target_language, max_retries=3, delay_seconds=2):
    translator = GoogleTranslator(source='auto', target=target_language)

    # Split content into chunks
    chunks = chunk_text(content, max_chars=4500)
    translated_chunks = []

    print(f"      Translating {len(chunks)} chunk(s)...", end='')

    for chunk_idx, chunk in enumerate(chunks):
        for attempt in range(max_retries):
            try:
                translated_text = translator.translate(chunk)
                translated_chunks.append(translated_text if translated_text is not None else chunk)
                break
            except Exception as e:
                print(f"\n      Translation error on chunk {chunk_idx + 1} (attempt {attempt + 1}/{max_retries}): {e}")
                time.sleep(delay_seconds * (attempt + 1))

                # If all retries failed
                if attempt == max_retries - 1:
                    print(f"      Failed to translate chunk {chunk_idx + 1}. Using original content.")
                    translated_chunks.append(chunk)

        # Small delay between chunks to avoid rate limiting
        if chunk_idx < len(chunks) - 1:
            time.sleep(0.5)

    print(" Done")
    return '\n'.join(translated_chunks)


# Get selected languages
selected_languages = [
    lang_code for lang_code, lang_data in language_checkboxes.items()
    if lang_data['widget'].value
]

# Ensure cloned_successfully exists (safe check)
if 'cloned_successfully' not in globals():
    print("'cloned_successfully' variable not found. Please run Cell 4 first.")
elif not selected_languages:
    print("No languages selected! Please select at least one language.")
elif not cloned_successfully:
    print("No repositories were successfully cloned!")
else:
    print(f"Translating {len(cloned_successfully)} repositories to {len(selected_languages)} languages\n")

    # Create output directory for translations
    translations_dir = Path("translations")
    translations_dir.mkdir(exist_ok=True)

    # Track translation results
    translation_results = {}

    for repo in cloned_successfully:
        print(f"\nProcessing repository: {repo}")
        repo_path = Path("cloned_repos") / repo

        # Get all text files
        text_files = get_all_text_files(repo_path)
        print(f"  Found {len(text_files)} text files")

        translation_results[repo] = {}

        for lang_code in selected_languages:
            lang_name = language_checkboxes[lang_code]['name']
            print(f"\n  Translating to {lang_name} ({lang_code})...")

            # Create language-specific directory
            lang_dir = translations_dir / repo / lang_code
            lang_dir.mkdir(parents=True, exist_ok=True)

            translated_count = 0
            skipped_count = 0

            for file_path in text_files:
                try:
                    # Read file content
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()

                    # Skip empty files
                    if not content.strip():
                        skipped_count += 1
                        continue

                    print(f"    Translating {file_path.name}...", end='')

                    # Translate content (now with chunking)
                    translated_content = translate_file_content(content, lang_code)

                    # Create output path maintaining directory structure
                    relative_path = file_path.relative_to(repo_path)
                    output_path = lang_dir / relative_path
                    output_path.parent.mkdir(parents=True, exist_ok=True)

                    # Save translated content
                    with open(output_path, 'w', encoding='utf-8') as f:
                        f.write(translated_content)

                    translated_count += 1
                    print(" Saved")

                except Exception as e:
                    print(f"\n    Error translating {file_path.name}: {str(e)}")
                    skipped_count += 1

            print(f"\n    Translated {translated_count} files")
            print(f"    Skipped {skipped_count} files")

            translation_results[repo][lang_code] = {
                'language': lang_name,
                'translated': translated_count,
                'skipped': skipped_count
            }

    # Save translation report
    report_path = translations_dir / "translation_report.json"
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump({
            'timestamp': datetime.now().isoformat(),
            'repositories': list(cloned_successfully),
            'languages': selected_languages,
            'results': translation_results
        }, f, indent=2)

    print("\nTRANSLATION COMPLETE!")
    print(f"Report saved to: {report_path}")

Translating 1 repositories to 1 languages


Processing repository: asi-protosymbiotic-signal
  Found 10 text files

  Translating to czech (cs)...
    Translating requirements.txt...      Translating 1 chunk(s)... Done
 Saved
    Translating package.json...      Translating 1 chunk(s)... Done
 Saved
    Translating README.md...      Translating 3 chunk(s)... Done
 Saved
    Translating asi_protosymbiotic_signal.json...      Translating 2 chunk(s)... Done
 Saved
    Translating Cargo.toml...      Translating 1 chunk(s)... Done
 Saved
    Translating setup.py...      Translating 2 chunk(s)... Done
 Saved
    Translating sha256hash-list.md...      Translating 1 chunk(s)... Done
 Saved
    Translating sha256hash-list.txt...      Translating 1 chunk(s)... Done
 Saved
    Translating asi_protosymbiotic_signal.yaml...      Translating 2 chunk(s)... Done
 Saved
    Translating asi_protosymbiotic_signal_pb2.py...      Translating 9 chunk(s)... Done
 Saved

    Translated 10 files
    Skipped 0 

In [9]:
# CELL 8 - Merge translations into single .txt files per language

from pathlib import Path

print(" Merging translated files into single .txt files per language...\n")

# Create merged output directory
merged_dir = Path("merged_translations")
merged_dir.mkdir(exist_ok=True)

for repo in cloned_successfully:
    print(f"\n Processing repository: {repo}")

    for lang_code in selected_languages:
        lang_name = language_checkboxes[lang_code]['name']
        print(f"  Merging {lang_name} ({lang_code}) files...")

        # Path to translated files
        lang_dir = Path("translations") / repo / lang_code

        if not lang_dir.exists():
            print(f"    No translations found for {lang_name}")
            continue

        # Collect all translated files
        all_files = sorted(lang_dir.rglob('*'))
        all_files = [f for f in all_files if f.is_file()]

        if not all_files:
            print(f"    No files to merge for {lang_name}")
            continue

        # Create merged file
        output_file = merged_dir / f"{repo}_{lang_code}.txt"

        with open(output_file, 'w', encoding='utf-8') as outfile:
            outfile.write(f"Repository: {repo}\n")
            outfile.write(f"Language: {lang_name} ({lang_code})\n")
            outfile.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            outfile.write("="*80 + "\n\n")

            for file_path in all_files:
                try:
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as infile:
                        content = infile.read()

                    # Write file header
                    relative_path = file_path.relative_to(lang_dir)
                    outfile.write(f"\n{'='*80}\n")
                    outfile.write(f"FILE: {relative_path}\n")
                    outfile.write(f"{'='*80}\n\n")

                    # Write content
                    outfile.write(content)
                    outfile.write("\n\n")

                except Exception as e:
                    print(f"    Error reading {file_path.name}: {str(e)}")

        print(f"    Merged {len(all_files)} files → {output_file.name}")

print("\n" + "="*80)
print("MERGING COMPLETE!")
print(f"Merged files saved to: {merged_dir}")
print("="*80)

# List all merged files
print("\n Generated files:")
for merged_file in sorted(merged_dir.glob("*.txt")):
    size_kb = merged_file.stat().st_size / 1024
    print(f"  • {merged_file.name} ({size_kb:.1f} KB)")

 Merging translated files into single .txt files per language...


 Processing repository: asi-protosymbiotic-signal
  Merging czech (cs) files...
    Merged 10 files → asi-protosymbiotic-signal_cs.txt

MERGING COMPLETE!
Merged files saved to: merged_translations

 Generated files:
  • asi-protosymbiotic-signal_cs.txt (77.2 KB)


In [11]:
# Cell 9
# Zip all translation files and upload to Google Drive

import zipfile
import os
from datetime import datetime
from google.colab import drive

# 1. Create timestamp for unique filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"asi_translations_{timestamp}.zip"

print("Creating zip file...")

# 2. Create zip file with all translations
output_dir = "translations"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add all translation files from the translations directory
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.join('translations', file)
            zipf.write(file_path, arcname)
            print(f"  ✔ Added to zip: {file}")

    # Also add the merged file if it exists
    merged_file = "all_translations_merged.txt"
    if os.path.exists(merged_file):
        zipf.write(merged_file, merged_file)
        print(f"  ✔ Added to zip: {merged_file}")

zip_size = os.path.getsize(zip_filename) / (1024*1024)
print(f"\n Zip file created: '{zip_filename}' ({zip_size:.2f} MB)")

# 3. Mount Google Drive
print("\n" + "=" * 60)
print("Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# 4. Copy zip file to Drive
drive_path = f"/content/drive/MyDrive/{zip_filename}"
print(f"\nUploading to Google Drive...")

import shutil
shutil.copy(zip_filename, drive_path)

print("SUCCESS!")
print(f"Zip file uploaded to: MyDrive/{zip_filename}")
print(f"Size: {zip_size:.2f} MB")
print("\nYou can find it in your Google Drive root folder!")

Creating zip file...
  ✔ Added to zip: translation_report.json
  ✔ Added to zip: requirements.txt
  ✔ Added to zip: package.json
  ✔ Added to zip: README.md
  ✔ Added to zip: asi_protosymbiotic_signal.json
  ✔ Added to zip: Cargo.toml
  ✔ Added to zip: setup.py
  ✔ Added to zip: sha256hash-list.md
  ✔ Added to zip: sha256hash-list.txt
  ✔ Added to zip: asi_protosymbiotic_signal.yaml
  ✔ Added to zip: asi_protosymbiotic_signal_pb2.py

 Zip file created: 'asi_translations_20260124_184604.zip' (0.02 MB)

Mounting Google Drive...
Mounted at /content/drive

Uploading to Google Drive...
SUCCESS!
Zip file uploaded to: MyDrive/asi_translations_20260124_184604.zip
Size: 0.02 MB

You can find it in your Google Drive root folder!
